# Laboratorio 6 Parte 1

### Reducción de dimensión: Selección de características

### 2019-I

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co

### Primer integrante:
Nombre: Alejandro Castaño Rojas

### Segundo integrante:
Nombre: Angélica Arroyave Mendoza

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

## Indicaciones

Este ejercicio tiene como objetivo implementar varias técnicas de selección de características y usar SVM para resolver un problema de clasificación multiclase.

Para el problema de clasificación usaremos la siguiente base de datos: https://archive.ics.uci.edu/ml/datasets/Cardiotocography


Analice la base de datos, sus características, su variable de salida y el contexto del problema.

Antes de iniciar a ejecutar las celdas, debe instalar la librería mlxtend que usaremos para los laboratorios de reducción de dimensión.
Para hacerlo solo tiene que usar el siguiente comando: sudo pip install mlxtend. También puede consultar la guía oficial de instalación
    de esta librería: https://rasbt.github.io/mlxtend/installation/

Analice y comprenda la siguiente celda de código donde se importan las librerías a usar y se carga la base de datos.

In [1]:
!pip install mlxtend

In [2]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import time

#cargamos la bd de entrenamiento
db = np.loadtxt('DB/DB_Fetal_Cardiotocograms.txt',delimiter='\t')  # Assuming tab-delimiter

X = db[:,0:22]

#Solo para dar formato a algunas variables
for i in range(1,7):
    X[:,i] = X[:,i]*1000

X = X
Y = db[:,22]

print('Dimensiones de la base de datos de entrenamiento. dim de X: ' + str(np.shape(X)) + '\tdim de Y: ' + str(np.shape(Y)))


Dimensiones de la base de datos de entrenamiento. dim de X: (2126, 22)	dim de Y: (2126,)


En la siguiente celda de código no tiene que completar nada. Analice, comprenda y ejecute el código y tenga en cuenta los resultados para completar la tabla que se le pide más abajo.

In [3]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

#Para calcular el costo computacional
tiempo_i = time.time()

#Creamos el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)

#Implemetamos la metodología de validación

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  

    #Aquí se entran y se valida el modelo sin hacer selección de características
    
    ######
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)
    
    #######

    Errores[j] = classification_error(ypred, y_test)
    j+=1

print("\nError de validación sin aplicar SFS: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print(('\n\nTiempo total de ejecución: ' + str(time.time()-tiempo_i)) + ' segundos.')


Error de validación sin aplicar SFS: 0.07712817787226504 +/- 0.05442325724156325


Tiempo total de ejecución: 1.6256515979766846 segundos.


## Ejercicio 1

1.1 Describa la metodología de validación que se está aplicando.

R/: La metodología de validación que se aplica es la Validación Cruzada con K folds


    
1.2 Con qué modelo se está resolviendo el problema planteado? Cuáles son los parámetros establecidos para el modelo?

R/:El modelo escogido fue máquinas de soporte vectorial. Los parámetros establecidos fueron: Kernel: RBF (Radial-basis function kernel), C: 100, función de decisión: ovr (One vs the rest), gamma: 0.0001.

## Ejercicio 2

En la siguiente celda, complete el código donde le sea indicado. Consulte la documentación oficial de la librería mlxtend para los métodos de selección de características. https://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/#sequential-feature-selector

In [6]:
#Feature Selection Function
#Recibe 4 parámetros: 1. el modelo (clf para nuestro caso), 2. el número de características final que se quiere alcanzar
#3. Si es forward (True), si es Backward False, 4. Si es es flotante (True), sino False
def select_features(modelo, n_features, fwd, fltg):
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring='accuracy',
           cv=0,
           n_jobs=-1)
    return sfs

def run_through(tecnique, n_features):
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Implemetamos la metodología de validación 

    Errores = np.ones(10)
    j = 0
    kf = KFold(n_splits=10)
    for train_index, test_index in kf.split(X):
        if tecnique == '':
            fwd = float('nan')
            floating = float('nan')
        elif tecnique == 'SFS':
            fwd = True
            floating = False
        elif tecnique == 'SBS':
            fwd = False
            floating = False
        elif tecnique == 'SFFS':
            fwd = True
            floating = True
        else: # elif tecnique == 'SBFS':
            fwd = False
            floating = True
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias


        #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
        #Ejercicio 3.1
        sf = select_features(clf, n_features=n_features, fwd=fwd, fltg=floating)

        #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
        #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
        sf = sf.fit(X_train, y_train)

        Errores[j] = 1-sf.k_score_
        j+=1
    error = np.mean(Errores)
    std = np.std(Errores)
    tiempo = time.time()-tiempo_i
    print("\nError de validación aplicando SFS: " + str(error) + " +/- " + str(std))
    print("\nEficiencia en validación aplicando SFS: " + str(sf.k_score_*100) + "%" )
    print ("\n\nTiempo total de ejecución: " + str(tiempo) + " segundos.")
    return error, std, tiempo, sf
    #print str(ypred)
    #print str(y_test)

## Ejercicio 3

3.1 En la celda de código anterior, varíe los parámetros correspondientes al número de características a seleccionar (use 3, 7 y 10) y la estrategia a implementar (SFS, SBS, SFFS, SBFS), para que complete la siguiente tabla de resultados:


In [5]:
import pandas as pd
import qgrid
tecnique_s = ['','SFS','SFS','SFS','SBS','SBS','SBS','SFFS','SFFS','SFFS','SBFS','SBFS','SBFS']
n_features_s = [22,3,7,10,3,7,10,3,7,10,3,7,10]
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['SVM sin selección','SVM + SFS','SVM + SFS','SVM + SFS','SVM + SBS','SVM + SBS','SVM + SBS','SVM + SFFS','SVM + SFFS','SVM + SFFS','SVM + SBFS','SVM + SBFS','SVM + SBFS']),
    '# de características seleccionadas' : pd.Series(n_features_s),
   })
df_types["Error de validación"] = ""
df_types["IC(std)"] = ""
df_types["Tiempo de ejecución"] = ""

df_types.set_index(['Tecnica','# de características seleccionadas'], inplace=True)
#df_types["Error de validación"][8] = "0.019"
#df_types["IC(std)"][8] = "0.002"
#df_types["Tiempo de ejecución"][8] = "107.9 s"
i = 0
for tecnique, n_features in zip(tecnique_s, n_features_s):
    error, std, tiempo, sf = run_through(tecnique, n_features)
    df_types["Error de validación"][i] = str(error)
    df_types["IC(std)"][i] = str(std)
    df_types["Tiempo de ejecución"][i] = str(tiempo)
    i += 1

#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    9.6s finished
Features: 1/22[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s finished
Features: 2/22[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.3s finished
Features: 3/22[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    0.6s fini


Error de validación aplicando SFS: 0.007003175217029612 +/- 0.0008487540566555215

Eficiencia en validación aplicando SFS: 99.32079414838036%


Tiempo total de ejecución: 283.73720264434814 segundos.


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    2.1s finished
Features: 1/3[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.8s finished
Features: 2/3[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.7s finished
Features: 3/3[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    2.5s finished
Features: 1/3[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.8s finished
Features: 2/3[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.9s finished
Features: 3/3[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  


Error de validación aplicando SFS: 0.04311674890112802 +/- 0.005501557850665672

Eficiencia en validación aplicando SFS: 95.61128526645768%


Tiempo total de ejecución: 45.650246143341064 segundos.


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    2.4s finished
Features: 1/7[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.8s finished
Features: 2/7[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.7s finished
Features: 3/7[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    0.6s finished
Features: 4/7[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    0.6s finished
Features: 5/7[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:    0.6s finished
Features: 6/7[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  

...

[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    0.9s finished
Features: 21/7[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s finished
Features: 20/7[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.1s finished
Features: 19/7[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent work


Error de validación aplicando SFS: 0.018866923284069137 +/- 0.002543983180019345

Eficiencia en validación aplicando SFS: 98.27586206896551%


Tiempo total de ejecución: 225.8110978603363 segundos.


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    0.9s finished
Features: 21/10[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s finished
Features: 20/10[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.1s finished
Features: 19/10[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent w


Error de validación aplicando SFS: 0.01102731079928836 +/- 0.0015918971954240022

Eficiencia en validación aplicando SFS: 98.90282131661442%


Tiempo total de ejecución: 168.2340168952942 segundos.


Error de validación  \
Tecnica           # de características seleccionadas                         
SVM sin selección 22                                  0.007003175217029612   
SVM + SFS         3                                    0.04311674890112802   
                  7                                   0.019911828052138435   
                  10                                   0.01306550189240313   
SVM + SBS         3                                   0.044214200697968754   
                  7                                    0.01991207385424808   
                  10                                  0.011288625753178627   
SVM + SFFS        3                                    0.04311674890112802   
                  7                                   0.019911828052138435   
                  10                                  0.012804132315821847   
SVM + SBFS        3                                   0.044475734142623125   
                  7                                   0.018866923284069137   
                  10                                   0.01102731079928836   

                                                                    IC(std)  \
Tecnica           # de características seleccionadas                          
SVM sin selección 22                                  0.0008487540566555215   
SVM + SFS         3                                    0.005501557850665672   
                  7                                    0.002727682121130486   
                  10                                   0.002347757559935403   
SVM + SBS         3                                    0.004748624592322402   
                  7                                   0.0026168019256843054   
                  10                                   0.001282860778208819   
SVM + SFFS        3                                    0.005501557850665672   
                  7                                    0.002727682121130486   
                  10                                  0.0022432590153498955   
SVM + SBFS        3                                    0.004775170586947885   
                  7                                    0.002543983180019345   
                  10                                  0.0015918971954240022   

                                                     Tiempo de ejecución  
Tecnica           # de características seleccionadas                      
SVM sin selección 22                                  283.73720264434814  
SVM + SFS         3                                   45.650246143341064  
                  7                                    91.07180881500244  
                  10                                   94.75584673881531  
SVM + SBS         3                                   122.94905257225037  
                  7                                   114.66388392448425  
                  10                                     96.788259267807  
SVM + SFFS        3                                    51.94139623641968  
                  7                                    95.81725835800171  
                  10                                  129.43679523468018  
SVM + SBFS        3                                   301.38518500328064  
                  7                                    225.8110978603363  
                  10                                   168.2340168952942

3.2 Según la teoría vista en el curso, se está usando una función tipo filtro o tipo wrapper y cuál es?

R/: La función utilizada es tipo wrapper dado que nuestro objetivo es un modelo de aprendizaje (svm)

3.3 Con los resultados de la tabla anterior haga un análisis de cuál es el mejor resultado teniendo en cuenta tanto la eficiencia en la clasificación como el costo computacional del modelo y la estrategia implementada.

R/:El mejor resultado es la técnica SBS con 10 características dando como resultado: Error=0.011288625753178627	y tiempo de ejecución =96.788259267807

In [7]:
error, std, tiempo, sf = run_through('SBS', 10)
sf.k_feature_idx_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   13.5s finished
Features: 21/10[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.9s finished
Features: 20/10[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.8s finished
Features: 19/10[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    0.8s finished
Features: 18/10[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    0.7s finished
Features: 17/10[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:    0.7s finished
Features: 16/10[Parallel(n_jobs=-1)]: Using


Error de validación aplicando SFS: 0.011288625753178627 +/- 0.001282860778208819

Eficiencia en validación aplicando SFS: 98.90282131661442%


Tiempo total de ejecución: 112.52936601638794 segundos.


(0, 1, 2, 4, 7, 9, 12, 18, 19, 21)

3.4 Haga uso del atributo sf.k_feature\_idx\_ (deje evidencia del código usado para esto) para identificar cuáles fueron las características seleccionadas en el mejor de los resultados encontrados. No presente los indices de las características sino sus nombres y descripción.

R/: Las características que dieron mejores resultados fueron:

LB - FHR baseline (beats per minute)
AC - # of accelerations per second
FM - # of fetal movements per second
DL - # of light decelerations per second
ASTV - percentage of time with abnormal short term variability
ALTV - percentage of time with abnormal long term variability
2Min - minimum of FHR histogram
Median - histogram median
Variance - histogram variance
CLASS - FHR pattern class code (1 to 10)

3.5 De acuerdo a los resultados encontrados y la respuesta anterior, usted como ingeniero de datos que le puede sugerir a un médico que esté trabajando en un caso enmarcado dentro del contexto de la base de datos trabajada, para que apoye su diagnóstico?

R/: Se le recomienda al médico prestar especial atención a las 10 características antes descritas, incluso si eso significa ignorar las otras 12 variables de las 22 totales. No obstante su diagnóstico puede ser apoyado por las variables que pueden resultar más familiares para su cotidianidad, como lo son LB, AC, FM y DL.